In [2]:
from glob import glob
import whisper
import librosa
import IPython.display as ipd
import numpy as np
import keras_nlp
import keras

2024-11-13 22:32:34.978955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731547955.045711  562475 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731547955.063276  562475 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-13 22:32:35.206823: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/gta/chiacchio/ProcVoz/nlp/env_nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Pleas

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""


In [4]:
screamModel = keras.models.load_model('/home/gta/chiacchio/ProcVoz/Insult-filter/models/scream_detection.keras')
emotionModel = keras.models.load_model('/home/gta/chiacchio/ProcVoz/Insult-filter/models/emotion_detection_2.keras')
transcriptModel = whisper.load_model("base")
classifier = keras_nlp.models.DistilBertClassifier.from_preset('distil_bert_base_multi', num_classes=2)


2024-11-13 22:32:37.193107: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-13 22:32:37.193126: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: paodeacucar
2024-11-13 22:32:37.193130: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: paodeacucar
2024-11-13 22:32:37.193155: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 565.57.1
2024-11-13 22:32:37.193169: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 565.57.1
2024-11-13 22:32:37.193172: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:259] kernel version seems to match DSO: 565.57.1
/home/gta/chiacchio/ProcVoz/nlp/env_nlp/lib/python3.10/site-packages/whisper/__init_

In [5]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print("GPUs available: ", gpus)

GPUs available:  []


In [74]:
sample_rate = 4000
trascribe_sample_rate = 16000
n_mfcc = 64


file = glob('/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio6.*')[0]
audio, _ = librosa.load(file, sr=sample_rate)
trascribe_audio, _ = librosa.load(file, sr=trascribe_sample_rate)

ipd.Audio(file)

In [70]:
sectionSizeSeconds = 10
audio_sections = [audio[i:i + sectionSizeSeconds*sample_rate]
                  for i in range(0, len(audio), sectionSizeSeconds*sample_rate)]
audio_sections[-1] = librosa.util.fix_length(audio_sections[-1], size=sample_rate*sectionSizeSeconds)

transcriptAudioSections = [trascribe_audio[i:i + sectionSizeSeconds*trascribe_sample_rate]
                  for i in range(0, len(trascribe_audio), sectionSizeSeconds*trascribe_sample_rate)]

transcriptAudioSections[-1] = librosa.util.fix_length(transcriptAudioSections[-1], size=trascribe_sample_rate*sectionSizeSeconds)

len(audio_sections)

1

In [71]:
# tst = glob('/home/gta/chiacchio/ProcVoz/data/screaming/*.wav')[0]
# file = glob('/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio7.*')[0]

# tst = librosa.load(file, sr=sample_rate)[0]
# tst = librosa.util.fix_length(tst, size=sample_rate*10)


tst = audio_sections[0]
transcribe_tst = transcriptAudioSections[0]
x = np.array(librosa.feature.mfcc(y=tst, sr=sample_rate, n_mfcc=n_mfcc))
x = np.expand_dims(x, axis=0)


print(screamModel.predict(x))
print(emotionModel.predict(x))
result = transcriptModel.transcribe(transcribe_tst, verbose=False, language="portuguese")['text']
print(result)
print(classifier.predict([result]))

# ipd.Audio(tst, rate=4000)

# emotionModel.input_shape


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[[9.9959415e-01 4.0591380e-04]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
[[7.7837106e-09 2.2211560e-07 5.3223643e-09 3.2270724e-05 9.9996710e-01
  3.2486440e-07]]


  0%|          | 0/1000 [00:00<?, ?frames/s]

100%|██████████| 1000/1000 [00:00<00:00, 1247.90frames/s]


 Vitória acordou com o latido dos cachorros no quintado vizinho. A sustada sentou na cama e olhou pela janela do quarto. No segundo andar da casa.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
[[0.06710286 0.13285983]]


In [80]:
glob('/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/*.*')[:-1]

['/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio3.m4a',
 '/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio5.waptt',
 '/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio2.m4a',
 '/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio4.waptt',
 '/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio7.waptt',
 '/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio6.waptt']

In [82]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module='librosa')
for file in glob('/home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/*.*')[:-1]:
    print('\n\nNew file: ', file)
    audio, _ = librosa.load(file, sr=sample_rate)
    trascribe_audio, _ = librosa.load(file, sr=trascribe_sample_rate)
    audio_sections = [audio[i:i + sectionSizeSeconds*sample_rate]
                    for i in range(0, len(audio), sectionSizeSeconds*sample_rate)]
    audio_sections[-1] = librosa.util.fix_length(audio_sections[-1], size=sample_rate*sectionSizeSeconds)

    transcriptAudioSections = [trascribe_audio[i:i + sectionSizeSeconds*trascribe_sample_rate]
                    for i in range(0, len(trascribe_audio), sectionSizeSeconds*trascribe_sample_rate)]

    transcriptAudioSections[-1] = librosa.util.fix_length(transcriptAudioSections[-1], size=trascribe_sample_rate*sectionSizeSeconds)

    for i in range(len(audio_sections)):
        # print(os.path.join(folder_path, audio_file))
        x = np.array(librosa.feature.mfcc(y=audio_sections[i], sr=sample_rate, n_mfcc=n_mfcc))
        x = np.expand_dims(x, axis=0)
        screamResult = screamModel.predict(x, verbose=0)
        emotionsResult = emotionModel.predict(x, verbose=0)
        transcription = transcriptModel.transcribe(transcriptAudioSections[i], verbose=0, language="portuguese")['text']
        nlpResult = classifier.predict([transcription], verbose=0)
        print(f"[{i*10} até {(i+1)*10}]: ", end=' ')
        print(f"Raiva + Nojo = {emotionsResult[0][2] + emotionsResult[0][3]}")
        print(f"Resultado Grito: {screamResult[0][0]} {screamResult[0][1]}")
        print(f"Resultado NLP: {nlpResult}")
        Block = False
        if(screamResult[0][0] < screamResult[0][1]):
            Block = True
            print("Bloquado por Grito")
        raivaNojo = emotionsResult[0][2] + emotionsResult[0][3]
        otherEmotions = sum(emotionsResult[0]) - raivaNojo
        if(raivaNojo > otherEmotions):
            Block = True
            print("Bloquado por Raiva ou Nojo")
        if(nlpResult[0][0] > nlpResult[0][1]):
            Block = True
            print("Bloquado por NLP", end=' ')
        print(f"Transcrição: {transcription}")
        print("Bloquear" if Block else "Liberar")




New file:  /home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio3.m4a


/tmp/ipykernel_562475/1294092552.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file, sr=sample_rate)
/home/gta/chiacchio/ProcVoz/nlp/env_nlp/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_562475/1294092552.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  trascribe_audio, _ = librosa.load(file, sr=trascribe_sample_rate)
/home/gta/chiacchio/ProcVoz/nlp/env_nlp/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/home/gta/chiacchio/ProcVoz/nlp/env_nlp/lib/python3.10/site-packages/whisper/transcribe.py:1

[0 até 10]:  Raiva + Nojo = 2.2996808992682694e-13
Resultado Grito: 0.9984917640686035 0.0015082771424204111
Resultado NLP: [[0.05977767 0.1246938 ]]
Transcrição:  O subsolo também produz golpes de estado, revoluções, histórias de espionagem e aventuras na floresta amazônica.
Liberar


New file:  /home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio5.waptt
[0 até 10]:  Raiva + Nojo = 3.227604611311108e-05
Resultado Grito: 0.9995941519737244 0.000405913800932467
Resultado NLP: [[0.06710286 0.13285983]]
Transcrição:  Vitória acordou com o latido dos cachorros no quintado vizinho. A sustada sentou na cama e olhou pela janela do quarto. No segundo andar da casa.
Liberar


New file:  /home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio2.m4a


/tmp/ipykernel_562475/1294092552.py:6: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file, sr=sample_rate)
/home/gta/chiacchio/ProcVoz/nlp/env_nlp/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/tmp/ipykernel_562475/1294092552.py:7: UserWarning: PySoundFile failed. Trying audioread instead.
  trascribe_audio, _ = librosa.load(file, sr=trascribe_sample_rate)
/home/gta/chiacchio/ProcVoz/nlp/env_nlp/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
/home/gta/chiacchio/ProcVoz/nlp/env_nlp/lib/python3.10/site-packages/whisper/transcribe.py:1

[0 até 10]:  Raiva + Nojo = 0.0007235119119286537
Resultado Grito: 0.9783387184143066 0.021661313250660896
Resultado NLP: [[0.05942304 0.14919698]]
Transcrição:  Castelas de açúcar sobre solos queimados de Cuba.
Liberar


New file:  /home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio4.waptt
[0 até 10]:  Raiva + Nojo = 0.9988653659820557
Resultado Grito: 0.9998582601547241 0.00014169169298838824
Resultado NLP: [[0.06733508 0.17685439]]
Bloquado por Raiva ou Nojo
Transcrição:  sem querer transforme por minha professora de iniciação à álgebra.
Bloquear
[10 até 20]:  Raiva + Nojo = 3.662374297164206e-07
Resultado Grito: 0.9996352195739746 0.0003647301346063614
Resultado NLP: [[0.05695776 0.13474438]]
Transcrição: 
Liberar


New file:  /home/gta/chiacchio/ProcVoz/Insult-filter/NPL/audios/audio7.waptt
[0 até 10]:  Raiva + Nojo = 0.9981316924095154
Resultado Grito: 0.868955135345459 0.13104479014873505
Resultado NLP: [[0.03146295 0.13800214]]
Bloquado por Raiva ou Nojo
Transcrição:  Va